In [6]:
import os
import time
import numpy as np
import xarray as xr

import pyarts
import FluxSimulator as fsm

In [8]:
idealized_sondes = "/work/mh0066/m301046/ml_clouds/sondes_for_radiation.nc"
min_wavenumber = 1
max_wavenumber = 3000
wave_bands = 10000

In [9]:
# Load dataset
ds = xr.open_dataset(idealized_sondes)
ds = ds.sortby("altitude", ascending=True)

In [10]:
# Convert xarray profiles to ARTS gridded fields
atms_grd = pyarts.arts.ArrayOfGriddedField4()
for i in range(ds.sonde.size):
    profile = ds.isel(sonde=i)

    profile_grd = fsm.generate_gridded_field_from_profiles(
        profile["p"].values,
        profile["t"].values,
        gases={
            "H2O": profile["H2O"],
            "CO2": profile["CO2"],
            "O3": profile["O3"],
            "N2": profile["N2"],
            "O2": profile["O2"],
        },
        z_field=profile["altitude"].values,
    )
    atms_grd.append(profile_grd)

In [12]:
# Setup Flux Simulator
gases = ["H2O", "CO2", "O2", "N2", "O3"]
f_grid = np.linspace(
    min_wavenumber, max_wavenumber, wave_bands
)  # frequency grid in cm^-1
f_grid_freq = pyarts.arts.convert.kaycm2freq(f_grid)  # converted to Hz
surface_reflectivity_lw = 0.05

species = [
    "H2O, H2O-SelfContCKDMT350, H2O-ForeignContCKDMT350",
    "O2-*-1e12-1e99,O2-CIAfunCKDMT100",
    "N2, N2-CIAfunCKDMT252, N2-CIArotCKDMT252",
    "CO2, CO2-CKDMT252",
    "O3",
    "O3-XFIT",
]

flux_simulator = fsm.FluxSimulator(f"100m_1000m_{wave_bands}_wavebands")
flux_simulator.ws.f_grid = f_grid_freq
flux_simulator.set_species(species)

In [13]:
flux_simulator.get_lookuptableBatch(
    atms_grd
)  # calculates lut for the current setup name and saves it to /cache. Try to delete /cache if this causes issues

LUT not found or does not fit.
 So, recalc...

...reading data

...setting up lut

...calculating lut

LUT calculation finished!


In [23]:
# Initialize output datasets
shape_flux = (len(ds.sonde), len(ds.altitude), len(f_grid))
shape_integrated = (len(ds.sonde), len(ds.altitude))

lw_fluxes = xr.Dataset(
    {
        "lw_flux_up_spectral": (
            ("sonde", "altitude", "f_grid"),
            np.full(shape_flux, np.nan),
        ),
        "lw_flux_down_spectral": (
            ("sonde", "altitude", "f_grid"),
            np.full(shape_flux, np.nan),
        ),
        "lw_flux_up": (("sonde", "altitude"), np.full(shape_integrated, np.nan)),
        "lw_flux_down": (("sonde", "altitude"), np.full(shape_integrated, np.nan)),
        "heating_rate": (("sonde", "altitude"), np.full(shape_integrated, np.nan)),
    },
    coords={
        "sonde_id": ("sonde", ds.sonde_id.values),
        "altitude": ("altitude", ds.altitude.values),
        "f_grid": ("f_grid", f_grid),
    },
)

In [24]:
lw_fluxes

<xarray.Dataset> Size: 31GB
Dimensions:                (sonde: 1069, altitude: 180, f_grid: 10000)
Coordinates:
    sonde_id               (sonde) <U8 34kB 'db2bef40' 'cfe7a063' ... 'ab47dc72'
  * altitude               (altitude) float64 1kB 0.0 50.0 ... 4.75e+04 4.85e+04
  * f_grid                 (f_grid) float64 80kB 1.0 1.3 1.6 ... 3e+03 3e+03
Dimensions without coordinates: sonde
Data variables:
    lw_flux_up_spectral    (sonde, altitude, f_grid) float64 15GB nan ... nan
    lw_flux_down_spectral  (sonde, altitude, f_grid) float64 15GB nan ... nan
    lw_flux_up             (sonde, altitude) float64 2MB nan nan nan ... nan nan
    lw_flux_down           (sonde, altitude) float64 2MB nan nan nan ... nan nan
    heating_rate           (sonde, altitude) float64 2MB nan nan nan ... nan nan

In [15]:
# Run simulation and store results
start_time = time.time()

for i in range(ds.sonde.size):
    prof = ds.isel(sonde=i)
    surface_temp = prof.isel(altitude=0)["t"].item()

    result = flux_simulator.flux_simulator_single_profile(
        atms_grd[i],
        surface_temp,
        0.0,
        surface_reflectivity=surface_reflectivity_lw,
        z_field=prof["altitude"],
    )

    # Store spectral and integrated fluxes
    lw_fluxes["lw_flux_up_spectral"].loc[dict(sonde=i)] = result[
        "spectral_flux_clearsky_up"
    ].T
    lw_fluxes["lw_flux_down_spectral"].loc[dict(sonde=i)] = result[
        "spectral_flux_clearsky_down"
    ].T
    lw_fluxes["lw_flux_up"].loc[dict(sonde=i)] = result["flux_clearsky_up"]
    lw_fluxes["lw_flux_down"].loc[dict(sonde=i)] = result["flux_clearsky_down"]
    lw_fluxes["heating_rate"].loc[dict(sonde=i)] = result["heating_rate_clearsky"]

    # ETA logging
    elapsed = time.time() - start_time
    remaining = elapsed / (i + 1) * (ds.sonde.size - i - 1)
    print(
        f"{i + 1}/{ds.sonde.size} complete | ETA: {remaining:.1f}s (~{remaining / 60:.1f} min)"
    )

setting up absorption...

...using stored LUT

starting calculation...



Start disort


1/1069 complete | ETA: 39063.3s (~651.1 min)
setting up absorption...

...using stored LUT

starting calculation...

2/1069 complete | ETA: 38182.8s (~636.4 min)
setting up absorption...



Start disort


...using stored LUT

starting calculation...

3/1069 complete | ETA: 38322.0s (~638.7 min)
setting up absorption...



...

33/1069 complete | ETA: 37223.4s (~620.4 min)
setting up absorption...

...using stored LUT

starting calculation...

34/1069 complete | ETA: 37170.4s (~619.5 min)
setting up absorption...



Start disort


...using stored LUT

starting calculation...



Start disort


KeyboardInterrupt: 

In [57]:
# update attributes
lw_fluxes.attrs.update({"DATA_PATH": DATA_PATH, **ds.attrs})

In [58]:
# --- Save results ---
if os.path.exists(OUTPUT_PATH):
    os.remove(OUTPUT_PATH)

lw_fluxes.to_netcdf(OUTPUT_PATH, mode="w")
print(f"Saved: {OUTPUT_PATH}")

Saved: flux_data.nc
